# Virtual Power Plant - Data Center Flexibility in ERCOT
**Course:** MAE/ENE 539  
**Model Type:** Linear Capacity Expansion with Demand-Side Flexibility  
**Dataset:** 16-Week ERCOT Representation (2,688 hours)

## 1. Setup and Dependencies

In [1]:
# Load required packages
using JuMP
using HiGHS
using DataFrames, CSV
using Statistics, LinearAlgebra
using Printf

# For reproducibility
import Random
Random.seed!(42)

Random.TaskLocalRNG()

## 2. Configuration and Scenario Setup

### Modify parameters to test different scenarios:

In [2]:
# Data Center Parameters
DATA_CENTER_MW = 1000.0   # Size of data center load (MW)
DATA_CENTER_ZONE = 1      # Location: 1 = West (Wind-rich), 2 = North/East (Demand center), 3 = South (Coast)
STRIKE_PRICE = 300.0      # $/MWh - Cost to curtail load (set to 9000 for "firm" load)

# Model Configuration
USE_FULL_YEAR = false                  # true = 52 weeks (8760 hrs), false = 16 weeks (2688 hrs)
ENABLE_RAMPING = false                 # Enable/disable ramping constraints for thermal units
ENABLE_STORAGE = true                  # Include battery storage options
ENABLE_TRANSMISSION_EXPANSION = true   # Allow new transmission capacity

# Analysis Options
RUN_SENSITIVITY_ANALYSIS = true   # Run multiple strike price scenarios
SAVE_HOURLY_RESULTS = true        # Export detailed hourly dispatch

# Display configuration
println("Scenario Configuration")
println("-" ^ 60)
println("Data Center: $(DATA_CENTER_MW) MW in Zone $(DATA_CENTER_ZONE)")
println("Strike Price: \$$(STRIKE_PRICE)/MWh")
println("Dataset: $(USE_FULL_YEAR ? "52 weeks (8,760 hrs)" : "16 weeks (2,688 hrs)")")
println("Ramping: $(ENABLE_RAMPING ? "Enabled" : "Disabled")")
println("Storage: $(ENABLE_STORAGE ? "Enabled" : "Disabled")")
println("Transmission Expansion: $(ENABLE_TRANSMISSION_EXPANSION ? "Enabled" : "Disabled")")

Scenario Configuration
------------------------------------------------------------
Data Center: 1000.0 MW in Zone 1
Strike Price: $300.0/MWh
Dataset: 16 weeks (2,688 hrs)
Ramping: Disabled
Storage: Enabled
Transmission Expansion: Enabled


## 3. Data Loading

Load generator characteristics, demand profiles, renewable availability, network topology, and fuel costs.

In [3]:
# Select data path
data_folder = USE_FULL_YEAR ? "52_weeks" : "16_weeks"
inputs_path = joinpath("ercot_brownfield_expansion", data_folder)

println("Loading data from: $(inputs_path)")

# Load generator data
generators = DataFrame(CSV.File(joinpath(inputs_path, "Generators_data.csv")))

# Select essential columns to reduce memory consumption
generators = select(generators, 
    :R_ID, :Resource, :zone, :THERM, :DISP, :NDISP, :STOR, :HYDRO, :RPS, :CES,
    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_size, :New_Build, :Max_Cap_MW,
    :Inv_cost_per_MWyr, :Fixed_OM_cost_per_MWyr, :Inv_cost_per_MWhyr, :Fixed_OM_cost_per_MWhyr,
    :Var_OM_cost_per_MWh, :Start_cost_per_MW, :Start_fuel_MMBTU_per_MW, :Heat_rate_MMBTU_per_MWh, :Fuel,
    :Min_power, :Ramp_Up_percentage, :Ramp_Dn_percentage, :Up_time, :Down_time,
    :Eff_up, :Eff_down
)

# Load demand, variability, fuels, and network
demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Load_data.csv")))
variability = DataFrame(CSV.File(joinpath(inputs_path, "Generators_variability.csv")))
variability = variability[:, 2:end]
fuels = DataFrame(CSV.File(joinpath(inputs_path, "Fuels_data.csv")))
network = DataFrame(CSV.File(joinpath(inputs_path, "Network.csv")))

# Summary statistics
println("Data Summary")
println("─" ^ 60)
println("Generators: $(nrow(generators)) resources")
println("Time steps: $(nrow(demand_inputs)) hours")
println("Zones: 3 (West, North/East, South/Coast)")
println("Transmission lines: $(nrow(network))")
println("Fuel types: $(nrow(fuels))")

Loading data from: ercot_brownfield_expansion\16_weeks
Data Summary
────────────────────────────────────────────────────────────
Generators: 55 resources
Time steps: 2688 hours
Zones: 3 (West, North/East, South/Coast)
Transmission lines: 3
Fuel types: 6


## 4. Data Center Integration

### Modeling Approach: "Virtual Generator"

We model the flexible data center with 2 components:
1. **Firm Load**: Added to demand in selected zone
2. **Virtual Generator**: Represents curtailment (load reduction), $VC = P_{strike}$

This formulation allows the optimizer to "dispatch" load reduction when marginal cost exceeds the strike price, without requiring binary variables.

In [4]:
# 1. Add flat load to demand profile
demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3)

# Add data center load to selected zone
zone_col = Symbol("Load_MW_z$(DATA_CENTER_ZONE)")
demand[!, zone_col] .+= DATA_CENTER_MW

println("Added $(DATA_CENTER_MW) MW flat load to Zone $(DATA_CENTER_ZONE)")

# Calculate zonal peak demand
peak_z1 = maximum(demand.Load_MW_z1)
peak_z2 = maximum(demand.Load_MW_z2)
peak_z3 = maximum(demand.Load_MW_z3)
total_peak = peak_z1 + peak_z2 + peak_z3

println("\nPeak Demands (with data center):")
println("  Zone 1 (West): $(round(peak_z1, digits=0)) MW")
println("  Zone 2 (North/East): $(round(peak_z2, digits=0)) MW")
println("  Zone 3 (South/Coast): $(round(peak_z3, digits=0)) MW")
println("  Total System: $(round(total_peak, digits=0)) MW")

# 2. Prepare generators DataFrame
generators.R_ID = string.(generators.R_ID)

# 3. Create virtual generator
new_dr = Dict(pairs(generators[1, :]))

# Configure DR resource parameters
new_dr[:R_ID] = "DataCenter_DR"
new_dr[:Resource] = "Demand_Response"
new_dr[:zone] = DATA_CENTER_ZONE
new_dr[:THERM] = 0
new_dr[:DISP] = 1
new_dr[:NDISP] = 0
new_dr[:STOR] = 0
new_dr[:HYDRO] = 0
new_dr[:RPS] = 0
new_dr[:CES] = 0
new_dr[:Existing_Cap_MW] = DATA_CENTER_MW
new_dr[:Max_Cap_MW] = DATA_CENTER_MW
new_dr[:New_Build] = 0
new_dr[:Var_OM_cost_per_MWh] = STRIKE_PRICE
new_dr[:Fuel] = "DR_Virtual"
new_dr[:Heat_rate_MMBTU_per_MWh] = 0.0
new_dr[:Min_power] = 0.0
new_dr[:Ramp_Up_percentage] = 1.0
new_dr[:Ramp_Dn_percentage] = 1.0
new_dr[:Inv_cost_per_MWyr] = 0.0
new_dr[:Fixed_OM_cost_per_MWyr] = 0.0
new_dr[:Start_cost_per_MW] = 0.0
new_dr[:Commit] = 0

# Add to generators list
push!(generators, new_dr)

println("Created virtual generator: 'DataCenter_DR'")
println("  Capacity: $(DATA_CENTER_MW) MW")
println("  Strike Price (Variable Cost): \$$(STRIKE_PRICE)/MWh")
println("  Zone: $(DATA_CENTER_ZONE)")

Added 1000.0 MW flat load to Zone 1

Peak Demands (with data center):
  Zone 1 (West): 1000.0 MW
  Zone 2 (North/East): 88086.0 MW
  Zone 3 (South/Coast): 5515.0 MW
  Total System: 94601.0 MW
Created virtual generator: 'DataCenter_DR'
  Capacity: 1000.0 MW
  Strike Price (Variable Cost): $300.0/MWh
  Zone: 1


## 5. Pre-Processing and Set Definitions

Calculate derived parameters, define optimization sets, and prepare data structures.

In [5]:
println("Pre-processing data")

# Process time/sample weights
hours_per_period = convert(Int64, demand_inputs.Hours_per_period[1])
P = convert(Array{Int64}, 1:demand_inputs.Subperiods[1])
W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
T = convert(Array{Int64}, demand_inputs.Time_index)

# Calculate sample weights for annualization
sample_weight = zeros(Float64, size(T, 1))
t = 1
for p in P
    for h in 1:hours_per_period
        if t <= length(T)
            sample_weight[t] = W[p] / hours_per_period
            t += 1
        end
    end
end

println("Time horizon: $(length(T)) hours")
println("Annualization factor: $(sum(sample_weight)) hrs/yr")

# Calculate VC (fuel + VOM)
generators.Var_Cost = zeros(Float64, nrow(generators))

for g in 1:nrow(generators)
    if generators.Fuel[g] == "DR_Virtual"
        # DR resource: cost is just the strike price
        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g]
    else
        # Normal generators: Fuel cost × Heat rate + VOM
        fuel_cost = fuels[fuels.Fuel .== generators.Fuel[g], :Cost_per_MMBtu][1]
        generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] + 
                                 fuel_cost * generators.Heat_rate_MMBTU_per_MWh[g]
    end
end

# Define sets
G = generators.R_ID  # All generators (including DR)
S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_segment)))
Z = convert(Array{Int64}, 1:3)  # Zones

# Network/Transmission
lines = select(network[1:2, :], :Network_lines, :z1, :z2, :z3, :Line_Max_Flow_MW, 
               :Line_Reinforcement_Cost_per_MW_yr)
lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr ./ 20
L = convert(Array{Int64}, lines.Network_lines)

# NSE segments
VOLL = demand_inputs.Voll[1]
nse = DataFrame(
    Segment = S,
    NSE_Cost = VOLL .* collect(skipmissing(demand_inputs.Cost_of_demand_curtailment_perMW)),
    NSE_Max = collect(skipmissing(demand_inputs.Max_demand_curtailment))
)

# Generator subsets
STOR = intersect(generators.R_ID[generators.STOR .>= 1], G)
NEW = intersect(generators.R_ID[generators.New_Build .== 1], G)
OLD = intersect(generators.R_ID[.!(generators.New_Build .== 1)], G)
THERM = intersect(generators.R_ID[generators.THERM .>= 1], G)

println("\nGenerator subsets:")
println("  Total resources (G): $(length(G))")
println("  Storage (STOR): $(length(STOR))")
println("  New-build candidates (NEW): $(length(NEW))")
println("  Existing units (OLD): $(length(OLD))")
println("  Thermal units (THERM): $(length(THERM))")

# Sync variability columns with generator IDs
num_original_gens = ncol(variability)
original_ids = generators.R_ID[1:num_original_gens]
rename!(variability, Symbol.(original_ids))

# Add DR availability
if !("DataCenter_DR" in names(variability))
    variability[!, "DataCenter_DR"] .= 1.0
end

# Create generator lookup dictionary
gen_lookup = Dict(row.R_ID => row for row in eachrow(generators))

println("Pre-processing complete")

Pre-processing data
Time horizon: 2688 hours
Annualization factor: 8736.0 hrs/yr

Generator subsets:
  Total resources (G): 56
  Storage (STOR): 3
  New-build candidates (NEW): 39
  Existing units (OLD): 17
  Thermal units (THERM): 20
Pre-processing complete


## 6. Optimization Model Formulation

### Model Structure

**Decision Variables:**
- Capacity: New builds, retirements, total capacity (generation, storage, transmission)
- Operations: Hourly generation, charging, state-of-charge, line flows
- Reliability: Non-served energy by segment and zone

**Objective:**
Minimize total annualized system cost ($ASC = Investment + OM_F + OM_V + NSE$)

**Constraints:**
- Energy balance (nodal)
- Generator capacity limits (renewable availability)
- Storage energy and power limits
- Transmission flow limits
- Ramping constraints (optional)
- Time-coupling (storage SOC evolution)

In [6]:
println("Building optimization model")

# Initialize model
Expansion_Model = Model(HiGHS.Optimizer)

# Solver settings for large problems
set_optimizer_attribute(Expansion_Model, "time_limit", 600.0)  # 10 min timeout
set_optimizer_attribute(Expansion_Model, "threads", 4)  # Use parallel threads

println("Model initialized")
println("  Time limit: 600 seconds")
println("  Threads: 4\n")

# Decision Variables
@variables(Expansion_Model, begin
    # Capacity variables
    vCAP[g in G] >= 0          # Total (MW)
    vNEW_CAP[g in NEW] >= 0    # New (MW)
    vRET_CAP[g in OLD] >= 0    # Retired (MW)
    
    # Storage energy capacity
    vE_CAP[g in STOR] >= 0     # Total (MWh)
    vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0
    vRET_E_CAP[g in intersect(STOR, OLD)] >= 0
    
    # Transmission capacity
    vT_CAP[l in L] >= 0       # Total line (MW)
    vNEW_T_CAP[l in L] >= 0   # New (MW)
    
    # Operational variables
    vGEN[T, G] >= 0         # Hourly generation (MW)
    vCHARGE[T, STOR] >= 0   # Hourly charging (MW)
    vSOC[T, STOR] >= 0      # State of charge (MWh)
    vNSE[T, S, Z] >= 0      # NSE (MW)
    vFLOW[T, L]             # Line flows (MW)
end)

println("Created decision variables")

# Constraints
println("\nAdding constraints...")

# 1. Nodal Energy Balance
@constraint(Expansion_Model, cDemandBalance[t in T, z in Z],
    sum(vGEN[t, g] for g in intersect(generators[generators.zone .== z, :R_ID], G)) +
    sum(vNSE[t, s, z] for s in S) - 
    sum(vCHARGE[t, g] for g in intersect(generators[generators.zone .== z, :R_ID], STOR)) -
    demand[t, z] - 
    sum(lines[l, Symbol(string("z", z))] * vFLOW[t, l] for l in L) == 0
)

println("  Demand balance ($(length(T) * length(Z)) constraints)")

# 2. Generator
for g in NEW
    if gen_lookup[g].Max_Cap_MW > 0
        set_upper_bound(vNEW_CAP[g], gen_lookup[g].Max_Cap_MW)
    end
end

# Capacity Accounting
@constraint(Expansion_Model, cCapOld[g in OLD],
    vCAP[g] == gen_lookup[g].Existing_Cap_MW - vRET_CAP[g]
)

@constraint(Expansion_Model, cCapNew[g in NEW],
    vCAP[g] == vNEW_CAP[g]
)

println("  Capacity accounting ($(length(G)) constraints)")

# 3. Generation Limits (includes renewable availability)
@constraint(Expansion_Model, cMaxPower[t in T, g in G],
    vGEN[t, g] <= variability[t, g] * vCAP[g]
)

println("  Generation limits ($(length(T) * length(G)) constraints)")

# 4. Storage
if ENABLE_STORAGE
    @constraint(Expansion_Model, cMaxCharge[t in T, g in STOR],
        vCHARGE[t, g] <= vCAP[g]
    )
    
    @constraint(Expansion_Model, cMaxSOC[t in T, g in STOR],
        vSOC[t, g] <= vE_CAP[g]
    )
    
    @constraint(Expansion_Model, cCapEnergyOld[g in intersect(STOR, OLD)],
        vE_CAP[g] == gen_lookup[g].Existing_Cap_MWh - vRET_E_CAP[g]
    )
    
    @constraint(Expansion_Model, cCapEnergyNew[g in intersect(STOR, NEW)],
        vE_CAP[g] == vNEW_E_CAP[g]
    )
    
    println("  Storage constraints ($(2*length(T)*length(STOR) + 2*length(STOR)) constraints)")
end

# 5. Tranmission
@constraint(Expansion_Model, cTransCap[l in L],
    vT_CAP[l] == lines.Line_Max_Flow_MW[l] + (ENABLE_TRANSMISSION_EXPANSION ? vNEW_T_CAP[l] : 0)
)

@constraint(Expansion_Model, cMaxFlow[t in T, l in L],
    vFLOW[t, l] <= vT_CAP[l]
)

@constraint(Expansion_Model, cMinFlow[t in T, l in L],
    vFLOW[t, l] >= -vT_CAP[l]
)

println("  Transmission constraints ($(length(L) + 2*length(T)*length(L)) constraints)")

# 6. Time-Coupling
STARTS = 1:hours_per_period:maximum(T)
INTERIORS = setdiff(T, STARTS)

# Ramping
if ENABLE_RAMPING
    @constraint(Expansion_Model, cRampUp[t in INTERIORS, g in THERM],
        vGEN[t, g] - vGEN[t-1, g] <= gen_lookup[g].Ramp_Up_percentage * vCAP[g]
    )
    
    @constraint(Expansion_Model, cRampDown[t in INTERIORS, g in THERM],
        vGEN[t-1, g] - vGEN[t, g] <= gen_lookup[g].Ramp_Dn_percentage * vCAP[g]
    )
    
    println("  Ramping constraints ($(2*length(INTERIORS)*length(THERM)) constraints)")
end

# Storage SOC evolution
if ENABLE_STORAGE
    @constraint(Expansion_Model, cSOC[t in INTERIORS, g in STOR],
        vSOC[t, g] == vSOC[t-1, g] + 
                      gen_lookup[g].Eff_up * vCHARGE[t, g] - 
                      vGEN[t, g] / gen_lookup[g].Eff_down
    )
    
    println("  Storage SOC evolution ($(length(INTERIORS)*length(STOR)) constraints)")
end

# Objective Function
@objective(Expansion_Model, Min,
    # Fixed O&M costs for all capacity
    sum(gen_lookup[g].Fixed_OM_cost_per_MWyr * vCAP[g] for g in G) +
    
    # Investment costs for new capacity
    sum(gen_lookup[g].Inv_cost_per_MWyr * vNEW_CAP[g] for g in NEW) +
    
    # Storage energy capacity costs
    sum(gen_lookup[g].Fixed_OM_cost_per_MWhyr * vE_CAP[g] for g in STOR) +
    sum(gen_lookup[g].Inv_cost_per_MWhyr * vNEW_E_CAP[g] for g in intersect(STOR, NEW)) +
    
    # Transmission costs
    sum(lines.Line_Fixed_Cost_per_MW_yr[l] * vT_CAP[l] for l in L) +
    (ENABLE_TRANSMISSION_EXPANSION ? 
        sum(lines.Line_Reinforcement_Cost_per_MW_yr[l] * vNEW_T_CAP[l] for l in L) : 0) +
    
    # Variable operating costs (fuel + VOM) - annualized
    sum(sample_weight[t] * gen_lookup[g].Var_Cost * vGEN[t, g] for t in T, g in G) +
    
    # NSE penalty
    sum(sample_weight[t] * nse.NSE_Cost[s] * vNSE[t, s, z] for t in T, s in S, z in Z)
)

println("\nObjective function formulated")
println("Model Statistics")
println("-" ^ 60)
println("Variables: $(num_variables(Expansion_Model))")
println("Constraints: $(num_constraints(Expansion_Model, count_variable_in_set_constraints=true))")

Building optimization model
Model initialized
  Time limit: 600 seconds
  Threads: 4

Created decision variables

Adding constraints...
  Demand balance (8064 constraints)
  Capacity accounting (56 constraints)
  Generation limits (150528 constraints)
  Storage constraints (16134 constraints)
  Transmission constraints (10754 constraints)
  Storage SOC evolution (8016 constraints)

Objective function formulated
Model Statistics
------------------------------------------------------------
Variables: 188282
Constraints: 376479


## 7. Solve Model

This cell solves the capacity expansion optimization problem. Depending on your hardware and configuration, this may take several minutes.

In [7]:
println("Solving Optimization Problem")
println("-" ^ 60)

@time optimize!(Expansion_Model)

# Check solution status
status = termination_status(Expansion_Model)
println("Solution Status: ", status)
println("-" ^ 60)

if status == MOI.OPTIMAL
    println("Optimal solution found!")
    obj_value = objective_value(Expansion_Model)
    println("\nTotal System Cost: \$$(round(obj_value / 1e9, digits=2)) billion/year")
elseif status == MOI.TIME_LIMIT
    println("Time limit reached. Solution may be suboptimal.")
    if has_values(Expansion_Model)
        obj_value = objective_value(Expansion_Model)
        println("Best solution found: \$$(round(obj_value / 1e9, digits=2)) billion/year")
    end
else
    println("Optimization failed or infeasible")
    println("Status: ", status)
end

Solving Optimization Problem
------------------------------------------------------------
Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 193549 rows; 188282 cols; 557248 nonzeros
Coefficient ranges:
  Matrix [1e-04, 1e+00]
  Cost   [1e+01, 9e+05]
  Bound  [8e+02, 9e+04]
  RHS    [1e+01, 9e+04]
Presolving model
175574 rows, 172994 cols, 521298 nonzeros  0s
Dependent equations search running on 16080 equations with time limit of 6.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.01s (limit = 6.00s)
175574 rows, 164930 cols, 513234 nonzeros  0s
Presolve : Reductions: rows 175574(-17975); columns 164930(-23352); elements 513234(-44014)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.1540369411e+07 Pr: 8064(4.19892e+07); Du: 0(2.23517e-09) 1s
      15026     1.6642878605e+09 Pr: 8790(6.29342e+07); Du: 0(1.49931e-06) 8s
      207

## 8. Results Analysis - Data Center Performance

### Key Metrics:
- **Curtailment Frequency**: How often is the data center load reduced?
- **Energy Shifted**: Total MWh of load curtailment (virtual generation)
- **Capacity Credit**: Does flexibility reduce peak capacity needs?
- **Economic Value**: Cost savings from demand response capability

In [8]:
if has_values(Expansion_Model)
    println("Data Center Flexibility Results")
    println("-" ^ 60)
    
    # Extract DR generation (curtailment)
    dc_gen = value.(vGEN)[:, "DataCenter_DR"]
    dc_gen_vec = [dc_gen[t] for t in T]
    
    # Calculate key metrics
    total_dc_curtailment_MWh = sum(sample_weight .* dc_gen_vec)
    total_dc_hours_curtailed = count(x -> x > 1.0, dc_gen_vec)
    max_curtailment_MW = maximum(dc_gen_vec)
    avg_curtailment_MW = total_dc_curtailment_MWh / sum(sample_weight[dc_gen_vec .> 1.0])
    
    # Calculate curtailment percentage
    total_possible_MWh = DATA_CENTER_MW * sum(sample_weight)
    curtailment_pct = (total_dc_curtailment_MWh / total_possible_MWh) * 100
    
    println("\nConfiguration:")
    println("  Data Center Size: $(DATA_CENTER_MW) MW")
    println("  Location: Zone $(DATA_CENTER_ZONE)")
    println("  Strike Price: \$$(STRIKE_PRICE)/MWh")
    
    println("\nCurtailment Statistics:")
    println("  Total Energy Curtailed: $(round(total_dc_curtailment_MWh, digits=0)) MWh/yr")
    println("  Curtailment as % of Load: $(round(curtailment_pct, digits=2))%")
    println("  Hours Curtailed: $(total_dc_hours_curtailed) hours ($(round(total_dc_hours_curtailed/length(T)*100, digits=2))%)")
    println("  Max Curtailment Event: $(round(max_curtailment_MW, digits=1)) MW")
    
    if total_dc_hours_curtailed > 0
        println("  Avg Curtailment When Active: $(round(avg_curtailment_MW, digits=1)) MW")
    end
    
    # Economic value
    curtailment_cost = total_dc_curtailment_MWh * STRIKE_PRICE
    println("\nEconomic Impact:")
    println("  Annual Curtailment Payments: \$$(round(curtailment_cost / 1e6, digits=2)) million/yr")
    
    # Find top 10 curtailment events
    curtailment_times = sortperm(dc_gen_vec, rev=true)[1:min(10, total_dc_hours_curtailed)]
    
    if total_dc_hours_curtailed > 0
        println("\nTop 10 Curtailment Events (Hour, MW):")
        for (i, t) in enumerate(curtailment_times)
            if dc_gen_vec[t] > 1.0
                println(@sprintf("  %2d. Hour %4d: %6.1f MW", i, t, dc_gen_vec[t]))
            end
        end
    end
else
    println("No solution available for analysis")
end

Data Center Flexibility Results
------------------------------------------------------------

Configuration:
  Data Center Size: 1000.0 MW
  Location: Zone 1
  Strike Price: $300.0/MWh

Curtailment Statistics:
  Total Energy Curtailed: 32255.0 MWh/yr
  Curtailment as % of Load: 0.37%
  Hours Curtailed: 23 hours (0.86%)
  Max Curtailment Event: 1000.0 MW
  Avg Curtailment When Active: 786.7 MW

Economic Impact:
  Annual Curtailment Payments: $9.68 million/yr

Top 10 Curtailment Events (Hour, MW):
   1. Hour  758: 1000.0 MW
   2. Hour  761: 1000.0 MW
   3. Hour  762: 1000.0 MW
   4. Hour 2538: 1000.0 MW
   5. Hour 2539: 1000.0 MW
   6. Hour 2556: 1000.0 MW
   7. Hour 2557: 1000.0 MW
   8. Hour 2558: 1000.0 MW
   9. Hour 2559: 1000.0 MW
  10. Hour 2560: 1000.0 MW


## 9. Results Analysis - Capacity Expansion

Analyze what new generation, storage, and transmission capacity the model selected.

In [9]:
if has_values(Expansion_Model)
    println("Capacity Expansion Results")
    println("-" ^ 60)
    
    # New Generation Capacity
    new_builds = DataFrame(
        ID = NEW,
        Resource = [gen_lookup[g].Resource for g in NEW],
        Zone = [gen_lookup[g].zone for g in NEW],
        New_MW = [value(vNEW_CAP[g]) for g in NEW],
        Inv_Cost_M = [value(vNEW_CAP[g]) * gen_lookup[g].Inv_cost_per_MWyr / 1e6 for g in NEW]
    )
    
    # Filter to significant builds (> 1 MW)
    new_builds = filter(row -> row.New_MW > 1.0, new_builds)
    sort!(new_builds, :New_MW, rev=true)
    
    if nrow(new_builds) > 0
        println("\nNew Generation Capacity:")
        println("  Total New Build: $(round(sum(new_builds.New_MW), digits=0)) MW")
        println("  Investment Cost: \$$(round(sum(new_builds.Inv_Cost_M), digits=1)) million/yr\n")
        show(new_builds, allrows=true, summary=false)
        
        # Summary by technology type
        println("\n\nNew Capacity by Technology:")
        tech_summary = combine(groupby(new_builds, :Resource), :New_MW => sum => :Total_MW)
        sort!(tech_summary, :Total_MW, rev=true)
        for row in eachrow(tech_summary)
            println(@sprintf("  %-30s: %8.0f MW", row.Resource, row.Total_MW))
        end
        
        # Summary by zone
        println("\nNew Capacity by Zone:")
        zone_summary = combine(groupby(new_builds, :Zone), :New_MW => sum => :Total_MW)
        sort!(zone_summary, :Zone)
        for row in eachrow(zone_summary)
            println(@sprintf("  Zone %d: %8.0f MW", row.Zone, row.Total_MW))
        end
    else
        println("\nNo new generation capacity built.")
    end
    
    # Retirements
    retirements = DataFrame(
        ID = OLD,
        Resource = [gen_lookup[g].Resource for g in OLD],
        Zone = [gen_lookup[g].zone for g in OLD],
        Retired_MW = [value(vRET_CAP[g]) for g in OLD]
    )
    
    retirements = filter(row -> row.Retired_MW > 1.0, retirements)
    
    if nrow(retirements) > 0
        println("\n\nRetirements:")
        println("  Total Retired: $(round(sum(retirements.Retired_MW), digits=0)) MW\n")
        show(retirements, allrows=true, summary=false)
    else
        println("\n\nNo existing capacity retired.")
    end
    
    # Storage
    if ENABLE_STORAGE && length(STOR) > 0
        stor_new = intersect(STOR, NEW)
        if length(stor_new) > 0
            storage_builds = DataFrame(
                ID = stor_new,
                Resource = [gen_lookup[g].Resource for g in stor_new],
                Zone = [gen_lookup[g].zone for g in stor_new],
                Power_MW = [value(vNEW_CAP[g]) for g in stor_new],
                Energy_MWh = [value(vNEW_E_CAP[g]) for g in stor_new],
                Duration_hrs = [value(vNEW_E_CAP[g]) / max(value(vNEW_CAP[g]), 0.001) for g in stor_new]
            )
            
            storage_builds = filter(row -> row.Power_MW > 1.0, storage_builds)
            
            if nrow(storage_builds) > 0
                println("\n\nNew Storage Capacity:")
                println("  Total Power: $(round(sum(storage_builds.Power_MW), digits=0)) MW")
                println("  Total Energy: $(round(sum(storage_builds.Energy_MWh), digits=0)) MWh\n")
                show(storage_builds, allrows=true, summary=false)
            end
        end
    end
    
    # Transmission
    if ENABLE_TRANSMISSION_EXPANSION
        trans_expansion = DataFrame(
            Line = L,
            Existing_MW = lines.Line_Max_Flow_MW,
            New_MW = [value(vNEW_T_CAP[l]) for l in L],
            Total_MW = [value(vT_CAP[l]) for l in L]
        )
        
        trans_expansion = filter(row -> row.New_MW > 1.0, trans_expansion)
        
        if nrow(trans_expansion) > 0
            println("\n\nTransmission Expansion:")
            show(trans_expansion, allrows=true, summary=false)
        else
            println("\n\nNo new transmission capacity built.")
        end
    end
end

Capacity Expansion Results
------------------------------------------------------------

New Generation Capacity:
  Total New Build: 25898.0 MW
  Investment Cost: $2083.2 million/yr

 Row │ ID      Resource              Zone   New_MW      Inv_Cost_M 
     │ String  String                Int64  Float64     Float64    
─────┼─────────────────────────────────────────────────────────────
   1 │ 38      utilitypv_losangeles      2  19057.2     1555.85
   2 │ 30      naturalgas_ccavgcf        2   6584.02     506.857
   3 │ 23      utilitypv_losangeles      1    157.848     12.5275
   4 │ 52      utilitypv_losangeles      3     98.4579     7.93091

New Capacity by Technology:
  utilitypv_losangeles          :    19314 MW
  naturalgas_ccavgcf            :     6584 MW

New Capacity by Zone:
  Zone 1:      158 MW
  Zone 2:    25641 MW
  Zone 3:       98 MW


Retirements:
  Total Retired: 1634.0 MW

 Row │ ID      Resource                    Zone   Retired_MW 
     │ String  String               

## 10. System-Level Metrics

Calculate reliability, emissions, and cost breakdown.

In [10]:
if has_values(Expansion_Model)
    println("System Performance Metrics")
    println("-" ^ 60)
    
    # Reliability
    nse_values = value.(vNSE)
    total_nse = sum(sample_weight[t] * sum(nse_values[t, s, z] for s in S, z in Z) for t in T)
    total_demand = sum(sample_weight .* [sum(demand[t, z] for z in Z) for t in T])
    
    println("\nReliability:")
    println("  Total Demand Served: $(round(total_demand / 1e6, digits=1)) TWh/yr")
    println("  Non-Served Energy: $(round(total_nse, digits=0)) MWh/yr")
    if total_nse > 0
        println("  Loss of Load Probability: $(round(total_nse / total_demand * 100, digits=4))%")
    else
        println("  Perfect reliability (no unserved energy)")
    end
    
    # Generation Mix
    gen_values = value.(vGEN)
    gen_by_resource = Dict{String, Float64}()
    
    for g in G
        resource_type = gen_lookup[g].Resource
        annual_gen = sum(sample_weight[t] * gen_values[t, g] for t in T)
        
        if haskey(gen_by_resource, resource_type)
            gen_by_resource[resource_type] += annual_gen
        else
            gen_by_resource[resource_type] = annual_gen
        end
    end
    
    total_generation = sum(values(gen_by_resource))
    
    println("\nGeneration Mix:")
    println("  Total Generation: $(round(total_generation / 1e6, digits=1)) TWh/yr\n")
    
    # Sort by generation amount
    sorted_resources = sort(collect(gen_by_resource), by=x->x[2], rev=true)
    
    for (resource, gen) in sorted_resources
        if gen > 100  # Only show resources with >100 MWh
            pct = gen / total_generation * 100
            println(@sprintf("  %-30s: %8.1f TWh (%5.2f%%)", 
                             resource, gen/1e6, pct))
        end
    end
    
    # Calculate renewable percentage
    renewable_gen = sum(gen_by_resource[r] for r in keys(gen_by_resource) 
                       if occursin("solar", lowercase(r)) || 
                          occursin("wind", lowercase(r)) ||
                          occursin("pv", lowercase(r)))
    
    println("\n  Renewable Penetration: $(round(renewable_gen / total_generation * 100, digits=1))%")
    
    # Cost Breakdown
    println("\nCost Breakdown:")
    
    # Calculate individual cost components
    fixed_om_cost = sum(gen_lookup[g].Fixed_OM_cost_per_MWyr * value(vCAP[g]) for g in G)
    inv_cost = sum(gen_lookup[g].Inv_cost_per_MWyr * value(vNEW_CAP[g]) for g in NEW)
    
    var_cost = sum(sample_weight[t] * gen_lookup[g].Var_Cost * gen_values[t, g] 
                  for t in T, g in G)
    
    nse_cost = sum(sample_weight[t] * nse.NSE_Cost[s] * nse_values[t, s, z] 
                  for t in T, s in S, z in Z)
    
    total_cost = objective_value(Expansion_Model)
    
    println(@sprintf("  Investment (Generation): \$%6.1f million/yr (%4.1f%%)",
                     inv_cost/1e6, inv_cost/total_cost*100))
    println(@sprintf("  Fixed O&M:              \$%6.1f million/yr (%4.1f%%)",
                     fixed_om_cost/1e6, fixed_om_cost/total_cost*100))
    println(@sprintf("  Variable O&M + Fuel:    \$%6.1f million/yr (%4.1f%%)",
                     var_cost/1e6, var_cost/total_cost*100))
    
    if nse_cost > 0
        println(@sprintf("  Unserved Energy:        \$%6.1f million/yr (%4.1f%%)",
                         nse_cost/1e6, nse_cost/total_cost*100))
    end
    
    println(@sprintf("\n  Total System Cost:      \$%6.1f billion/yr",
                     total_cost/1e9))
    
    # Average cost per MWh
    avg_cost_per_mwh = total_cost / total_demand
    println(@sprintf("  Average Cost:           \$%6.2f/MWh", avg_cost_per_mwh))
end

System Performance Metrics
------------------------------------------------------------

Reliability:
  Total Demand Served: 463.5 TWh/yr
  Non-Served Energy: 32438.0 MWh/yr
  Loss of Load Probability: 0.007%

Generation Mix:
  Total Generation: 463.5 TWh/yr

  natural_gas_fired_combined_cycle:    125.3 TWh (27.03%)
  conventional_steam_coal       :    116.6 TWh (25.15%)
  onshore_wind_turbine          :     65.8 TWh (14.20%)
  naturalgas_ccavgcf            :     57.5 TWh (12.40%)
  utilitypv_losangeles          :     52.8 TWh (11.39%)
  nuclear                       :     43.9 TWh ( 9.46%)
  solar_photovoltaic            :      1.2 TWh ( 0.25%)
  natural_gas_fired_combustion_turbine:      0.4 TWh ( 0.10%)
  small_hydroelectric           :      0.1 TWh ( 0.02%)
  Demand_Response               :      0.0 TWh ( 0.01%)

  Renewable Penetration: 25.8%

Cost Breakdown:
  Investment (Generation): $2083.2 million/yr (14.5%)
  Fixed O&M:              $3169.0 million/yr (22.0%)
  Variable O&M +

## 11. Sensitivity Analysis (Optional)

Run multiple scenarios with different strike prices to create a value curve for demand flexibility.

In [11]:
if RUN_SENSITIVITY_ANALYSIS
    println("Strike Price Sensitivity Analysis")
    println("-" ^ 60)

    # Define strike price sweep
    strike_prices = [100.0, 200.0, 300.0, 500.0, 1000.0, 2000.0, 5000.0, 9000.0]

    # Results storage
    sensitivity_results = DataFrame(
        StrikePrice = Float64[],
        TotalCost = Float64[],
        Curtailment_MWh = Float64[],
        Curtailment_Pct = Float64[],
        Curtailment_Hours = Int64[],
        NewCapacity_MW = Float64[],
        NewSolar_MW = Float64[],
        NewGas_MW = Float64[],
        NewStorage_MW = Float64[],
        RenewablePct = Float64[],
        SolveTime = Float64[],
        Status = String[]
    )

    # Helper function to build and solve model
    function solve_scenario(strike_price::Float64, gen_lookup_local, generators_local)
        # Update DR resource cost
        gen_lookup_local["DataCenter_DR"].Var_Cost = strike_price

        # Build model
        model = Model(HiGHS.Optimizer)
        set_optimizer_attribute(model, "time_limit", 600.0)
        set_optimizer_attribute(model, "threads", 4)
        set_optimizer_attribute(model, "output_flag", false)  # Suppress solver output

        # Decision variables
        @variables(model, begin
            vCAP[g in G] >= 0
            vNEW_CAP[g in NEW] >= 0
            vRET_CAP[g in OLD] >= 0
            vE_CAP[g in STOR] >= 0
            vNEW_E_CAP[g in intersect(STOR, NEW)] >= 0
            vRET_E_CAP[g in intersect(STOR, OLD)] >= 0
            vT_CAP[l in L] >= 0
            vNEW_T_CAP[l in L] >= 0
            vGEN[T, G] >= 0
            vCHARGE[T, STOR] >= 0
            vSOC[T, STOR] >= 0
            vNSE[T, S, Z] >= 0
            vFLOW[T, L]
        end)

        # Constraints
        # 1. Demand Balance
        @constraint(model, cDemandBalance[t in T, z in Z],
            sum(vGEN[t, g] for g in intersect(generators_local[generators_local.zone .== z, :R_ID], G)) +
            sum(vNSE[t, s, z] for s in S) -
            sum(vCHARGE[t, g] for g in intersect(generators_local[generators_local.zone .== z, :R_ID], STOR)) -
            demand[t, z] -
            sum(lines[l, Symbol(string("z", z))] * vFLOW[t, l] for l in L) == 0
        )

        # 2. Capacity limits
        for g in NEW
            if gen_lookup_local[g].Max_Cap_MW > 0
                set_upper_bound(vNEW_CAP[g], gen_lookup_local[g].Max_Cap_MW)
            end
        end

        @constraint(model, cCapOld[g in OLD],
            vCAP[g] == gen_lookup_local[g].Existing_Cap_MW - vRET_CAP[g]
        )

        @constraint(model, cCapNew[g in NEW],
            vCAP[g] == vNEW_CAP[g]
        )

        # 3. Generation limits
        @constraint(model, cMaxPower[t in T, g in G],
            vGEN[t, g] <= variability[t, g] * vCAP[g]
        )

        # 4. Storage
        if ENABLE_STORAGE
            @constraint(model, cMaxCharge[t in T, g in STOR],
                vCHARGE[t, g] <= vCAP[g]
            )

            @constraint(model, cMaxSOC[t in T, g in STOR],
                vSOC[t, g] <= vE_CAP[g]
            )

            @constraint(model, cCapEnergyOld[g in intersect(STOR, OLD)],
                vE_CAP[g] == gen_lookup_local[g].Existing_Cap_MWh - vRET_E_CAP[g]
            )

            @constraint(model, cCapEnergyNew[g in intersect(STOR, NEW)],
                vE_CAP[g] == vNEW_E_CAP[g]
            )
        end

        # 5. Transmission
        @constraint(model, cTransCap[l in L],
            vT_CAP[l] == lines.Line_Max_Flow_MW[l] + (ENABLE_TRANSMISSION_EXPANSION ? vNEW_T_CAP[l] : 0)
        )

        @constraint(model, cMaxFlow[t in T, l in L],
            vFLOW[t, l] <= vT_CAP[l]
        )

        @constraint(model, cMinFlow[t in T, l in L],
            vFLOW[t, l] >= -vT_CAP[l]
        )

        # 6. Time coupling
        if ENABLE_RAMPING
            @constraint(model, cRampUp[t in INTERIORS, g in THERM],
                vGEN[t, g] - vGEN[t-1, g] <= gen_lookup_local[g].Ramp_Up_percentage * vCAP[g]
            )

            @constraint(model, cRampDown[t in INTERIORS, g in THERM],
                vGEN[t-1, g] - vGEN[t, g] <= gen_lookup_local[g].Ramp_Dn_percentage * vCAP[g]
            )
        end

        if ENABLE_STORAGE
            @constraint(model, cSOC[t in INTERIORS, g in STOR],
                vSOC[t, g] == vSOC[t-1, g] +
                              gen_lookup_local[g].Eff_up * vCHARGE[t, g] -
                              vGEN[t, g] / gen_lookup_local[g].Eff_down
            )
        end

        # Objective
        @objective(model, Min,
            sum(gen_lookup_local[g].Fixed_OM_cost_per_MWyr * vCAP[g] for g in G) +
            sum(gen_lookup_local[g].Inv_cost_per_MWyr * vNEW_CAP[g] for g in NEW) +
            sum(gen_lookup_local[g].Fixed_OM_cost_per_MWhyr * vE_CAP[g] for g in STOR) +
            sum(gen_lookup_local[g].Inv_cost_per_MWhyr * vNEW_E_CAP[g] for g in intersect(STOR, NEW)) +
            sum(lines.Line_Fixed_Cost_per_MW_yr[l] * vT_CAP[l] for l in L) +
            (ENABLE_TRANSMISSION_EXPANSION ?
                sum(lines.Line_Reinforcement_Cost_per_MW_yr[l] * vNEW_T_CAP[l] for l in L) : 0) +
            sum(sample_weight[t] * gen_lookup_local[g].Var_Cost * vGEN[t, g] for t in T, g in G) +
            sum(sample_weight[t] * nse.NSE_Cost[s] * vNSE[t, s, z] for t in T, s in S, z in Z)
        )

        # Solve
        solve_start = time()
        optimize!(model)
        solve_time = time() - solve_start

        return model, solve_time
    end

    # Run sensitivity analysis
    for (idx, sp) in enumerate(strike_prices)
        println("[$idx/$(length(strike_prices))] Testing Strike Price: \$$(sp)/MWh...")

        # Create a deep copy of gen_lookup for this scenario
        gen_lookup_scenario = deepcopy(gen_lookup)
        generators_scenario = deepcopy(generators)

        try
            # Solve the scenario
            scenario_model, solve_time = solve_scenario(sp, gen_lookup_scenario, generators_scenario)

            # Check solution status
            status = termination_status(scenario_model)

            if has_values(scenario_model)
                # Extract results
                total_cost = objective_value(scenario_model)

                # Curtailment metrics
                dc_gen_vec = [value(scenario_model[:vGEN][t, "DataCenter_DR"]) for t in T]
                curtailment_mwh = sum(sample_weight .* dc_gen_vec)
                curtailment_pct = (curtailment_mwh / (DATA_CENTER_MW * sum(sample_weight))) * 100
                curtailment_hours = count(x -> x > 1.0, dc_gen_vec)

                # Capacity expansion metrics
                new_cap_total = sum(value(scenario_model[:vNEW_CAP][g]) for g in NEW)

                # New capacity by technology
                new_solar = 0.0
                new_gas = 0.0
                new_storage = 0.0

                for g in NEW
                    cap = value(scenario_model[:vNEW_CAP][g])
                    resource = gen_lookup_scenario[g].Resource

                    if occursin("pv", lowercase(resource)) || occursin("solar", lowercase(resource))
                        new_solar += cap
                    elseif occursin("gas", lowercase(resource))
                        new_gas += cap
                    end
                end

                # Storage
                if ENABLE_STORAGE
                    for g in intersect(STOR, NEW)
                        new_storage += value(scenario_model[:vNEW_CAP][g])
                    end
                end

                # Generation mix for renewable percentage
                gen_by_res = Dict{String, Float64}()
                for g in G
                    resource = gen_lookup_scenario[g].Resource
                    annual_gen = sum(sample_weight[t] * value(scenario_model[:vGEN][t, g]) for t in T)

                    if haskey(gen_by_res, resource)
                        gen_by_res[resource] += annual_gen
                    else
                        gen_by_res[resource] = annual_gen
                    end
                end

                total_gen = sum(values(gen_by_res))
                renewable_gen = sum(gen_by_res[r] for r in keys(gen_by_res)
                                   if occursin("solar", lowercase(r)) ||
                                      occursin("wind", lowercase(r)) ||
                                      occursin("pv", lowercase(r)))
                renewable_pct = (renewable_gen / total_gen) * 100

                # Store results
                push!(sensitivity_results, (
                    StrikePrice = sp,
                    TotalCost = total_cost,
                    Curtailment_MWh = curtailment_mwh,
                    Curtailment_Pct = curtailment_pct,
                    Curtailment_Hours = curtailment_hours,
                    NewCapacity_MW = new_cap_total,
                    NewSolar_MW = new_solar,
                    NewGas_MW = new_gas,
                    NewStorage_MW = new_storage,
                    RenewablePct = renewable_pct,
                    SolveTime = solve_time,
                    Status = string(status)
                ))

                println("  Solved in $(round(solve_time, digits=1))s - Cost: \$$(round(total_cost/1e9, digits=2))B - Curtailment: $(round(curtailment_mwh, digits=0)) MWh")
            else
                println("  No solution found - Status: $(status)")
                push!(sensitivity_results, (
                    StrikePrice = sp,
                    TotalCost = NaN,
                    Curtailment_MWh = NaN,
                    Curtailment_Pct = NaN,
                    Curtailment_Hours = 0,
                    NewCapacity_MW = NaN,
                    NewSolar_MW = NaN,
                    NewGas_MW = NaN,
                    NewStorage_MW = NaN,
                    RenewablePct = NaN,
                    SolveTime = solve_time,
                    Status = string(status)
                ))
            end
        catch e
            println("  Error: $(e)")
            push!(sensitivity_results, (
                StrikePrice = sp,
                TotalCost = NaN,
                Curtailment_MWh = NaN,
                Curtailment_Pct = NaN,
                Curtailment_Hours = 0,
                NewCapacity_MW = NaN,
                NewSolar_MW = NaN,
                NewGas_MW = NaN,
                NewStorage_MW = NaN,
                RenewablePct = NaN,
                SolveTime = 0.0,
                Status = "ERROR"
            ))
        end
    end

    # Display results
    println("Sensitivity Analysis Results")
    println("-" ^ 60)
    show(sensitivity_results, allrows=true)

    # Export results
    mkpath("results")
    CSV.write("results/sensitivity_analysis.csv", sensitivity_results)
    println("\n\n✓ Results exported to results/sensitivity_analysis.csv")

    # Summary
    println("Key Insights")
    println("─" ^ 60)

    valid_results = filter(row -> !isnan(row.TotalCost), sensitivity_results)

    if nrow(valid_results) > 0
        # Find optimal strike price range
        min_cost_row = valid_results[argmin(valid_results.TotalCost), :]
        max_curtail_row = valid_results[argmax(valid_results.Curtailment_MWh), :]

        println("Lowest System Cost:")
        println("  Strike Price: \$$(min_cost_row.StrikePrice)/MWh")
        println("  Total Cost: \$$(round(min_cost_row.TotalCost/1e9, digits=2)) billion/yr")
        println("  Curtailment: $(round(min_cost_row.Curtailment_MWh, digits=0)) MWh/yr ($(round(min_cost_row.Curtailment_Pct, digits=2))%)")

        println("\nMaximum Flexibility Utilization:")
        println("  Strike Price: \$$(max_curtail_row.StrikePrice)/MWh")
        println("  Curtailment: $(round(max_curtail_row.Curtailment_MWh, digits=0)) MWh/yr ($(round(max_curtail_row.Curtailment_Pct, digits=2))%)")
        println("  Curtailment Hours: $(max_curtail_row.Curtailment_Hours)")

        # Capacity value analysis
        if nrow(valid_results) >= 2
            firm_cost = valid_results[valid_results.StrikePrice .== maximum(valid_results.StrikePrice), :TotalCost][1]
            flex_cost = valid_results[valid_results.StrikePrice .== minimum(valid_results.StrikePrice), :TotalCost][1]
            savings = firm_cost - flex_cost

            println("\nCapacity Value of Flexibility:")
            println("  System cost w/ firm load (\$9000/MWh): \$$(round(firm_cost/1e9, digits=2))B/yr")
            println("  System cost w/ flexible load (\$$(minimum(valid_results.StrikePrice))/MWh): \$$(round(flex_cost/1e9, digits=2))B/yr")
            println("  Annual savings: \$$(round(savings/1e6, digits=1)) million/yr")
        end
    end
else
    println("\n Sensitivity analysis disabled (set RUN_SENSITIVITY_ANALYSIS = true to enable)")
end

Strike Price Sensitivity Analysis
------------------------------------------------------------
[1/8] Testing Strike Price: $100.0/MWh...
  Solved in 170.6s - Cost: $14.39B - Curtailment: 34327.0 MWh
[2/8] Testing Strike Price: $200.0/MWh...
  Solved in 156.2s - Cost: $14.4B - Curtailment: 32625.0 MWh
[3/8] Testing Strike Price: $300.0/MWh...
  Solved in 208.5s - Cost: $14.4B - Curtailment: 32255.0 MWh
[4/8] Testing Strike Price: $500.0/MWh...
  Solved in 180.9s - Cost: $14.41B - Curtailment: 24362.0 MWh
[5/8] Testing Strike Price: $1000.0/MWh...
  Solved in 183.3s - Cost: $14.41B - Curtailment: 0.0 MWh
[6/8] Testing Strike Price: $2000.0/MWh...
  Solved in 190.2s - Cost: $14.41B - Curtailment: 0.0 MWh
[7/8] Testing Strike Price: $5000.0/MWh...
  Solved in 205.6s - Cost: $14.41B - Curtailment: 0.0 MWh
[8/8] Testing Strike Price: $9000.0/MWh...
  Solved in 197.9s - Cost: $14.41B - Curtailment: 0.0 MWh
Sensitivity Analysis Results
----------------------------------------------------------

## 12. Export Results (Optional)

Export detailed results to CSV files for further analysis in Excel, Python, or visualization tools.

In [12]:
if has_values(Expansion_Model) && SAVE_HOURLY_RESULTS
    println("\nExporting results to CSV files...")
    
    # Create results directory
    results_dir = "results"
    mkpath(results_dir)
    
    # 1. Export capacity expansion decisions
    capacity_results = DataFrame(
        Generator = G,
        Resource = [gen_lookup[g].Resource for g in G],
        Zone = [gen_lookup[g].zone for g in G],
        Existing_MW = [gen_lookup[g].Existing_Cap_MW for g in G],
        Total_MW = [value(vCAP[g]) for g in G],
        New_MW = [g in NEW ? value(vNEW_CAP[g]) : 0.0 for g in G],
        Retired_MW = [g in OLD ? value(vRET_CAP[g]) : 0.0 for g in G]
    )
    
    CSV.write(joinpath(results_dir, "capacity_expansion.csv"), capacity_results)
    println("  Saved capacity_expansion.csv")
    
    # 2. Export hourly generation (WARNING: Large file!)
    gen_hourly = DataFrame(
        Hour = repeat(T, outer=length(G)),
        Generator = repeat(G, inner=length(T)),
        Generation_MW = vec([value(vGEN[t, g]) for t in T, g in G])
    )
    
    CSV.write(joinpath(results_dir, "hourly_generation.csv"), gen_hourly)
    println("  Saved hourly_generation.csv")
    
    # 3. Export data center curtailment events
    dc_gen_vec = [value(vGEN[t, "DataCenter_DR"]) for t in T]
    curtailment_events = DataFrame(
        Hour = T,
        Curtailment_MW = dc_gen_vec,
        Demand_Z1 = demand.Load_MW_z1,
        Demand_Z2 = demand.Load_MW_z2,
        Demand_Z3 = demand.Load_MW_z3
    )
    
    CSV.write(joinpath(results_dir, "datacenter_curtailment.csv"), curtailment_events)
    println("  Saved datacenter_curtailment.csv")
    
    # 4. Export summary statistics
    summary = DataFrame(
        Metric = [
            "Total_Cost_per_yr",
            "DataCenter_Size_MW",
            "DataCenter_Zone",
            "Strike_Price_per_MWh",
            "Curtailment_MWh_per_yr",
            "Curtailment_Hours",
            "New_Capacity_MW",
            "Total_Generation_TWh"
        ],
        Value = [
            objective_value(Expansion_Model),
            DATA_CENTER_MW,
            DATA_CENTER_ZONE,
            STRIKE_PRICE,
            sum(sample_weight .* dc_gen_vec),
            count(x -> x > 1.0, dc_gen_vec),
            sum(new_builds.New_MW),
            sum(values(gen_by_resource)) / 1e6
        ]
    )
    
    CSV.write(joinpath(results_dir, "summary.csv"), summary)
    println("  Saved summary.csv")
    
    println("\n All results exported to '$(results_dir)/' directory")
else
    println("\n Result export disabled (set SAVE_HOURLY_RESULTS = true to enable)")
end


Exporting results to CSV files...
  Saved capacity_expansion.csv
  Saved hourly_generation.csv
  Saved datacenter_curtailment.csv
  Saved summary.csv

 All results exported to 'results/' directory
